<a href="https://colab.research.google.com/github/GKCSV/Resilia_ProjetoFinal_M6/blob/GK/Analise_de_despesas_publicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**ANÁLISE DE DESPESAS PÚBLICAS GERADAS POR DEPUTADOS NOS ANOS DE 2019, 2020 E 2021**

#**1. Preparação dos dados para consultas:**

##1.1. Inicialização do ambiente e importação dos dados:

In [92]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from matplotlib.ticker import StrMethodFormatter
from sklearn.preprocessing import MinMaxScaler

In [93]:
despesas2019 = pd.read_csv('https://raw.githubusercontent.com/GKCSV/Resilia_ProjetoFinal_M6/main/Ano-2019.csv', sep = ';')
despesas2020 = pd.read_csv('https://raw.githubusercontent.com/GKCSV/Resilia_ProjetoFinal_M6/main/Ano-2020.csv', sep = ';')
despesas2021 = pd.read_csv('https://raw.githubusercontent.com/GKCSV/Resilia_ProjetoFinal_M6/main/Ano-2021.csv', sep = ';')

##1.2. Formatação do dataset:

###1.2.1. Mesclagem dos DataFrames para um único:

In [94]:
despesas1920 = pd.merge(despesas2019, despesas2020, how='outer')
despesas_gerais = pd.merge(despesas1920, despesas2021, how='outer')

###1.2.2. Visualizando as colunas presentes:

In [95]:
despesas_gerais.columns.tolist()

['txNomeParlamentar',
 'cpf',
 'ideCadastro',
 'nuCarteiraParlamentar',
 'nuLegislatura',
 'sgUF',
 'sgPartido',
 'codLegislatura',
 'numSubCota',
 'txtDescricao',
 'numEspecificacaoSubCota',
 'txtDescricaoEspecificacao',
 'txtFornecedor',
 'txtCNPJCPF',
 'txtNumero',
 'indTipoDocumento',
 'datEmissao',
 'vlrDocumento',
 'vlrGlosa',
 'vlrLiquido',
 'numMes',
 'numAno',
 'numParcela',
 'txtPassageiro',
 'txtTrecho',
 'numLote',
 'numRessarcimento',
 'vlrRestituicao',
 'nuDeputadoId',
 'ideDocumento',
 'urlDocumento']

###1.2.3 Exclusão de colunas não-utilizadas através do comando .drop():
###Serão removidas as colunas:

'cpf'

'nuCarteiraParlamentar'

'nuLegislatura'

'codLegislatura'

'numSubCota'

'numEspecificacaoSubCota'

'txtNumero'

'indTipoDocumento'

'numParcela'

'txtTrecho'

'numLote'

'numRessarcimento'

In [96]:
excluir_colunas = despesas_gerais.drop(columns=['cpf', 'nuCarteiraParlamentar', 'nuLegislatura', 'codLegislatura', 'numSubCota', 'numEspecificacaoSubCota', 'txtNumero', 'indTipoDocumento', 'vlrDocumento', 'vlrGlosa', 'numParcela', 'txtTrecho', 'numLote', 'numRessarcimento', 'vlrRestituicao'])


###1.2.4. Criando cópia do dataset para atualização de nomeação:

In [97]:
despesas_selecionadas = excluir_colunas.copy()

###1.2.5 Visualizando os tipos de dados presentes nas colunas:

In [98]:
despesas_selecionadas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 555500 entries, 0 to 555499
Data columns (total 16 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   txNomeParlamentar          555500 non-null  object 
 1   ideCadastro                553759 non-null  float64
 2   sgUF                       553759 non-null  object 
 3   sgPartido                  553759 non-null  object 
 4   txtDescricao               555500 non-null  object 
 5   txtDescricaoEspecificacao  136177 non-null  object 
 6   txtFornecedor              555467 non-null  object 
 7   txtCNPJCPF                 531710 non-null  object 
 8   datEmissao                 539850 non-null  object 
 9   vlrLiquido                 555500 non-null  float64
 10  numMes                     555500 non-null  int64  
 11  numAno                     555500 non-null  int64  
 12  txtPassageiro              129493 non-null  object 
 13  nuDeputadoId               55

###1.2.6. Corrigindo o formato da data:

In [99]:
despesas_selecionadas['datEmissao'] = pd.to_datetime(despesas_selecionadas['datEmissao'], format = '%Y/%m/%d')

###1.2.7. Exibindo a contagem de dados:

In [100]:
despesas_selecionadas.count()

txNomeParlamentar            555500
ideCadastro                  553759
sgUF                         553759
sgPartido                    553759
txtDescricao                 555500
txtDescricaoEspecificacao    136177
txtFornecedor                555467
txtCNPJCPF                   531710
datEmissao                   539850
vlrLiquido                   555500
numMes                       555500
numAno                       555500
txtPassageiro                129493
nuDeputadoId                 555500
ideDocumento                 555500
urlDocumento                 349639
dtype: int64

###1.2.8 Criando o DataFrame do Estado de MS:

In [101]:
despesas_ms = despesas_selecionadas.loc[despesas_selecionadas['sgUF']=='MS']

###1.2.9. Criando o DataFrame das Lideranças Partidárias:

In [102]:
liderancas_partidarias = despesas_selecionadas[despesas_selecionadas['sgUF'].isnull()]
liderancas_partidarias

,txNomeParlamentar,ideCadastro,sgUF,sgPartido,txtDescricao,txtDescricaoEspecificacao,txtFornecedor,txtCNPJCPF,datEmissao,vlrLiquido,numMes,numAno,txtPassageiro,nuDeputadoId,ideDocumento,urlDocumento
0,LID.GOV-CD,NaN,NaN,NaN,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,NaN,CRISTIANE FERREIRA EPP,037.139.030/0010-8,2019-01-16,111.30,1,2019,NaN,2812,6738121,http://camara.leg.br/cota-parlamentar/nota-fis...
1,LID.GOV-CD,NaN,NaN,NaN,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,NaN,CRISTIANE FERREIRA EPP,037.139.030/0010-8,2019-02-04,117.19,2,2019,NaN,2812,6769606,http://camara.leg.br/cota-parlamentar/nota-fis...
2,LID.GOV-CD,NaN,NaN,NaN,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,NaN,CRISTIANE FERREIRA EPP,037.139.030/0010-8,2019-02-25,120.15,2,2019,NaN,2812,6771099,http://camara.leg.br/cota-parlamentar/nota-fis...
3,LID.GOV-CD,NaN,NaN,NaN,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,NaN,CRISTIANE FERREIRA EPP,037.139.030/0010-8,2019-03-28,151.18,3,2019,NaN,2812,6793690,http://camara.leg.br/cota-parlamentar/nota-fis...
4,LID.GOV-CD,NaN,NaN,NaN,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,NaN,PANIFICADORA FERREIRA NETO LTDA - EPP,037.139.030/0010-8,2019-05-07,119.90,5,2019,NaN,2812,6821674,http://camara.leg.br/cota-parlamentar/nota-fis...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450334,LIDMIN,NaN,NaN,NaN,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,NaN,LEONARDO AUGUSTO RODRIGUES 04995267166,272.646.780/0013-8,2021-08-10,4000.00,7,2021,NaN,2806,7237514,https://www.camara.leg.br/cota-parlamentar/doc...
450335,LIDMIN,NaN,NaN,NaN,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,NaN,BISCOITOS CASEIROS HOMONNAI LTDA,046.448.200/0017-7,2021-02-24,137.97,2,2021,NaN,2806,7157995,http://camara.leg.br/cota-parlamentar/nota-fis...
450336,LIDMIN,NaN,NaN,NaN,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,NaN,R.E.M. BISCOITOS CASEIROS LTDA,012.175.080/0010-9,2021-02-01,175.18,2,2021,NaN,2806,7147536,http://camara.leg.br/cota-parlamentar/nota-fis...
450337,LIDMIN,NaN,NaN,NaN,LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES,NaN,PANTANAL VEÍCULOS LTDA,073.193.230/0019-1,2021-02-25,3800.00,2,2021,NaN,2806,7160332,https://www.camara.leg.br/cota-parlamentar/doc...


###1.2.10. Renomeando colunas:

In [103]:
despesas_ms.rename(columns={'txNomeParlamentar':'Nome-Parlamentar', 'ideCadastro':'ID-Parlamentar', 'sgUF':'Estado', 'sgPartido':'Partido', 'txtDescricao':'Despesa-Descrição', 'txtDescricaoEspecificacao':'Despesa-Especificação', 'txtFornecedor':'Nome-Fornecedor', 'txtCNPJCPF':'CNPJ/CPF-Fornecedor', 'datEmissao':'Emissão-NF', 'vlrLiquido':'Valor Liquido-NF', 'numMes':'Mês', 'numAno':'Ano', 'txtPassageiro':'Passageiro', 'nuDeputadoId':'ID-Parlamentar-Compra', 'ideDocumento':'Numero-NF', 'urlDocumento':'URL-NF'})

,Nome-Parlamentar,ID-Parlamentar,Estado,Partido,Despesa-Descrição,Despesa-Especificação,Nome-Fornecedor,CNPJ/CPF-Fornecedor,Emissão-NF,Valor Liquido-NF,Mês,Ano,Passageiro,ID-Parlamentar-Compra,Numero-NF,URL-NF
29077,Geraldo Resende,74374.0,MS,PSDB,TELEFONIA,NaN,RAMAL,000.000.000/0000-6,NaT,1.40,7,2019,NaN,1649,0,NaN
29078,Vander Loubet,74376.0,MS,PT,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,NaN,aguas guarirobas,040.895.700/0015-0,2019-01-14,77.42,1,2019,NaN,1697,6759317,https://www.camara.leg.br/cota-parlamentar/doc...
29079,Vander Loubet,74376.0,MS,PT,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,NaN,ENERGISA MS,154.138.260/0015-0,2019-01-11,249.98,1,2019,NaN,1697,6759322,https://www.camara.leg.br/cota-parlamentar/doc...
29080,Vander Loubet,74376.0,MS,PT,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,NaN,ronaldo guedine ribeiro,120.251.668/85 -,2019-02-07,2150.00,1,2019,NaN,1697,6760118,https://www.camara.leg.br/cota-parlamentar/doc...
29081,Vander Loubet,74376.0,MS,PT,COMBUSTÍVEIS E LUBRIFICANTES.,Veículos Automotores,ALPHAVILE COMERCIO DE COMBUSTIVEIS LTDA,103.396.620/0014-4,2019-01-06,100.00,1,2019,NaN,1697,6759290,https://www.camara.leg.br/cota-parlamentar/doc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553674,Bia Cavassa,207176.0,MS,PSDB,PASSAGEM AÉREA - RPA,NaN,Cia Aérea - TAM,020.128.620/0016-0,2021-02-04,1618.57,2,2021,Bia Cavassa,3445,1745718,NaN
553675,Bia Cavassa,207176.0,MS,PSDB,PASSAGEM AÉREA - RPA,NaN,Cia Aérea - TAM,020.128.620/0016-0,2021-02-05,1085.47,2,2021,Bia Cavassa,3445,1745744,NaN
553676,Bia Cavassa,207176.0,MS,PSDB,PASSAGEM AÉREA - RPA,NaN,Cia Aérea - TAM,020.128.620/0016-0,2021-02-10,2337.33,2,2021,Bia Cavassa,3445,1745819,NaN
553677,Bia Cavassa,207176.0,MS,PSDB,PASSAGEM AÉREA - RPA,NaN,Cia Aérea - TAM,020.128.620/0016-0,2021-04-30,131.47,4,2021,Bia Cavassa,3445,1749934,NaN


###1.2.11. Visualização dos DataFrames para consultas:

In [104]:
despesas_ms.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7970 entries, 29077 to 553678
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   txNomeParlamentar          7970 non-null   object        
 1   ideCadastro                7970 non-null   float64       
 2   sgUF                       7970 non-null   object        
 3   sgPartido                  7970 non-null   object        
 4   txtDescricao               7970 non-null   object        
 5   txtDescricaoEspecificacao  2519 non-null   object        
 6   txtFornecedor              7970 non-null   object        
 7   txtCNPJCPF                 7712 non-null   object        
 8   datEmissao                 7724 non-null   datetime64[ns]
 9   vlrLiquido                 7970 non-null   float64       
 10  numMes                     7970 non-null   int64         
 11  numAno                     7970 non-null   int64         
 12  

In [105]:
despesas_ms.columns.tolist()

['txNomeParlamentar',
 'ideCadastro',
 'sgUF',
 'sgPartido',
 'txtDescricao',
 'txtDescricaoEspecificacao',
 'txtFornecedor',
 'txtCNPJCPF',
 'datEmissao',
 'vlrLiquido',
 'numMes',
 'numAno',
 'txtPassageiro',
 'nuDeputadoId',
 'ideDocumento',
 'urlDocumento']

In [106]:
liderancas_partidarias.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1741 entries, 0 to 450338
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   txNomeParlamentar          1741 non-null   object        
 1   ideCadastro                0 non-null      float64       
 2   sgUF                       0 non-null      object        
 3   sgPartido                  0 non-null      object        
 4   txtDescricao               1741 non-null   object        
 5   txtDescricaoEspecificacao  151 non-null    object        
 6   txtFornecedor              1741 non-null   object        
 7   txtCNPJCPF                 1741 non-null   object        
 8   datEmissao                 1741 non-null   datetime64[ns]
 9   vlrLiquido                 1741 non-null   float64       
 10  numMes                     1741 non-null   int64         
 11  numAno                     1741 non-null   int64         
 12  txtP

In [107]:
liderancas_partidarias.columns.tolist()

['txNomeParlamentar',
 'ideCadastro',
 'sgUF',
 'sgPartido',
 'txtDescricao',
 'txtDescricaoEspecificacao',
 'txtFornecedor',
 'txtCNPJCPF',
 'datEmissao',
 'vlrLiquido',
 'numMes',
 'numAno',
 'txtPassageiro',
 'nuDeputadoId',
 'ideDocumento',
 'urlDocumento']

#**2. Exportando os DataFrames "depesas_ms" e "liderancas_partidarias" para arquivos .xlsx:**

###**Aqui serão gerados dois arquivos .xlsx a serem utilizado no Power BI:**

In [108]:
with pd.ExcelWriter('despesas_ms.xlsx') as writer:  
    despesas_ms.to_excel(writer, sheet_name='MS')

In [109]:
with pd.ExcelWriter('liderancas_partidarias.xlsx') as writer:  
    liderancas_partidarias.to_excel(writer, sheet_name='MS')

#**3. Análise dos dados:**

###2.1. - Qual o maior valor que um parlamentar gastou com um único fornecedor? Este fornecedor em específico possui alguma ligação com o parlamentar identificado?

###2.2. - Qual o maior valor gasto em uma única despesa de alimentação? Existe alguma explicação plausível?

###2.3. - Qual o percentual de notas fiscais indisponíveis para consulta? Nesta relação está inclusa alguma categoria que possa vir a configurar fraude?

###2.4 - De possíveis fraudes por falta de prestação de contas, qual o top 5 parlamentares?

###2.5 - Qual o parlamentar mais caro para o governo se considerado suas despesas pessoais e as despesas a partir de lideranças partidárias?

###2.6 - Existe alguma despesa duplicada entre parlamentar x liderança partidária?